In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
def dump_json(test_data,output_json_name=None):
    if output_json_name is not None:
            with open(output_json_name, "w",encoding='utf8') as output_file:
                json.dump(test_data,output_file,ensure_ascii=False)

def get_json(path):
    with open(path,"r", encoding='utf8') as f:
        data=json.load(f)
    return data

In [3]:
import itertools

def findcomb(s,n):
    return itertools.combinations(s,n)

def findper(s,n):
    return itertools.permutations(s,n)

def checkd(d):
    subs = findper(range(len(d)),3)
    for a,b,c in subs:
        # print(a,b,c)
        if d[a][b]>d[b][c]+d[c][a]:
            return False
    return True

In [4]:
# data = get_json('/content/drive/MyDrive/Academic/IT4663/6_28_impossible_data.json')
data = get_json('/content/drive/MyDrive/Academic/IT4663/6_26_new_data.json')+\
        get_json('/content/drive/MyDrive/Academic/IT4663/6_28_impossible_data.json')

In [59]:
import time
from tqdm.auto import tqdm

min_val = 1e9
min_path=0

def satisfy(path,itloc,item2take):
    for i in range(1,len(item2take)):
        total = sum([itloc[i][s] for s in path if s!=0])
        if total<item2take[i]:
            return False
    return True

def calc_path(path,d):
    return sum([d[path[i]][path[i+1]] for i in range(len(path)-1)])

def dfs_branch(dt,lim=300,cut=True):
    st_time=time.time()
    n,m,itloc = dt['n'],dt['m'],dt['itloc'] # num item, num shelves, itloc -> [item, shelve]
    d,itemavail,item2take = dt['d'],dt['itemavail'],dt['item2take']
    global min_path
    global min_val
    min_path = list(range(0,m+1))+[0]
    min_val = calc_path(min_path,d)
    min_d = min([i for y in d for i in y if i>0])
    visited = []
    queue = []  
    
    tmp_min, tmp_path = visit([],set(),itloc,item2take,d,
                            min_val,min_path,st_time,lim,min_d,cut)
    min_val=tmp_min
    min_path=[0]+tmp_path+[0]
    
    return min_val, min_path, time.time()-st_time

def visit(path,visited,itloc,item2take,d,
          prev_min,prev_min_path,st_time,lim,min_d,cut):
    global min_path
    global min_val
    # if time.time()-st_time>lim:
    #     return min_val,min_path
    if cut:
        if calc_path([0]+path,d)+min_d>=min_val:
            return min_val, min_path
    if satisfy(path,itloc,item2take):            
        tmp_min = calc_path([0]+path+[0],d)
        if tmp_min<min_val:
            min_val=tmp_min
            min_path=path
            print(min_val)
    # next = max(1,path[-1])
    for j in range(1,len(d)):
        if j not in visited:
            tmp_min, tmp_path = visit(path+[j],visited.union({j}),itloc,item2take,d,
                                        min_val,min_path,st_time,lim,min_d,cut)
            if tmp_min<min_val:
                min_val = tmp_min
                min_path= tmp_path
    return min_val, min_path

In [61]:
import sys
sys.setrecursionlimit(3500)
solutions = []
for i in range(len(data)):
    # DFS
    sol = dfs_branch(data[i],3600,False)
    # DFS - branch&bound
    sol = dfs_branch(data[i],3600,True)
    solutions.append(sol)
    print(sol)
    print(data[i]['optimal_value'])
    dump_json(solutions,'/content/drive/MyDrive/Academic/IT4663/Results/dfs.json')

In [ ]:
len(solutions)

22